<a href="https://colab.research.google.com/github/Sreevatsan03hub/CLOUD-COMPUTING/blob/main/RestaurantSalesPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Load the dataset (UPDATE the file path based on your Google Drive)
file_path = "/content/drive/MyDrive/restaurant_sales_data.csv"
df = pd.read_csv(file_path)

# Convert Date to Datetime Format
df['Date'] = pd.to_datetime(df['Date'])
df['DayOfWeek'] = df['Date'].dt.dayofweek  # Monday=0, Sunday=6

# One-Hot Encoding for Weather and Events
encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
encoded_features = encoder.fit_transform(df[['Weather', 'Event']])
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['Weather', 'Event']))
df = pd.concat([df, encoded_df], axis=1).drop(['Weather', 'Event', 'Date'], axis=1)

# Splitting Features and Target Variables
X = df.drop(columns=['Idly', 'Poori', 'Vada', 'Dosa', 'Chapathi'])  # Features
y = df[['Idly', 'Poori', 'Vada', 'Dosa', 'Chapathi']]  # Target Variables

# Scale the Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split Data into Train and Test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define the Neural Network Model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(5, activation='linear')  # 5 output neurons for the food items
])

# Compile the Model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the Model
model.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_test, y_test), verbose=1)

# --- Predicting Tomorrow's Sales ---
# Example: Tomorrow's conditions
tomorrow_features = {
    "DayOfWeek": [3],  # Example: Assuming tomorrow is Thursday
    "Weather_Sunny": [1],
    "Weather_Rainy": [0],
    "Weather_Cloudy": [0],
    "Event_None": [1],
    "Event_Festival": [0],
    "Event_Holiday": [0]
}

# Convert into DataFrame
tomorrow_df = pd.DataFrame(tomorrow_features)

# Ensure missing columns are handled
for col in X.columns:
    if col not in tomorrow_df.columns:
        tomorrow_df[col] = 0  # Add missing columns with default value

# Reorder columns to match training data
tomorrow_df = tomorrow_df[X.columns]

# Scale the Features
tomorrow_scaled = scaler.transform(tomorrow_df)

# Predict Tomorrow's Sales
predicted_sales = model.predict(tomorrow_scaled)
predicted_sales = np.round(predicted_sales).astype(int)  # Round to whole numbers

# Create a DataFrame for Output
predicted_sales_df = pd.DataFrame(predicted_sales, columns=['Idly', 'Poori', 'Vada', 'Dosa', 'Chapathi'])

# Display Predictions
print("\n📌 **Predicted Sales for Tomorrow:**")
print(predicted_sales_df)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 88ms/step - loss: 7228.0371 - mae: 80.4307 - val_loss: 6981.8750 - val_mae: 78.9899
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 7338.6304 - mae: 80.5520 - val_loss: 6922.2700 - val_mae: 78.6366
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 7092.1733 - mae: 79.5035 - val_loss: 6849.5620 - val_mae: 78.2004
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - loss: 6954.5752 - mae: 78.6229 - val_loss: 6753.2559 - val_mae: 77.6169
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 6795.7681 - mae: 77.7852 - val_loss: 6624.4565 - val_mae: 76.8235
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 6883.5591 - mae: 77.9180 - val_loss: 6445.5234 - val_mae: 75.6986
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 6737.2690 - mae: 77.4727 - val_loss: 6211.0493 - val_mae: 74.1820
Epoch 8/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 6265.5742 - mae: 74.6600 - val_loss: 5912.6699 - val_mae: 72.1883
Epoch 9/